<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Finding the perfect neighborhood in Manhattan</font></h1>

## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in New York City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [4]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

/bin/sh: wget: command not found
Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [6]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [7]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [8]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [10]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [12]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [14]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


#### Create a map of New York with neighborhoods superimposed on top.

In [15]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [16]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [17]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [18]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = '3XQ3ZRFJHG1ZHFW0PVBKPUNQO0YFN1AGGEVFY3O5YGEQSI4M' # your Foursquare ID
CLIENT_SECRET = 'NHK22QAHONYF1BZ2XILVEB4BXVTQMWBE5N1RGCTBY2VE2F1X' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3XQ3ZRFJHG1ZHFW0PVBKPUNQO0YFN1AGGEVFY3O5YGEQSI4M
CLIENT_SECRET:NHK22QAHONYF1BZ2XILVEB4BXVTQMWBE5N1RGCTBY2VE2F1X


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [20]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

Get the neighborhood's latitude and longitude values.

In [21]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [174]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=3XQ3ZRFJHG1ZHFW0PVBKPUNQO0YFN1AGGEVFY3O5YGEQSI4M&client_secret=NHK22QAHONYF1BZ2XILVEB4BXVTQMWBE5N1RGCTBY2VE2F1X&v=20180604&ll=40.87655077879964,-73.91065965862981&radius=1000&limit=100'

Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [175]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1fd21cdb04f53ab554c2f4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 79,
  'suggestedBounds': {'ne': {'lat': 40.88555078779965,
    'lng': -73.89877901972629},
   'sw': {'lat': 40.86755076979963, 'lng': -73.92254029753333}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [176]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [177]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Sam's Pizza,Pizza Place,40.879435,-73.905859
4,Loeser's Delicatessen,Sandwich Place,40.879242,-73.905471


And how many venues were returned by Foursquare?

In [178]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

79 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [185]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [186]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [187]:
print(manhattan_venues.shape)
manhattan_venues.head(30)

(3979, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Sam's Pizza,40.879435,-73.905859,Pizza Place
4,Marble Hill,40.876551,-73.91066,Loeser's Delicatessen,40.879242,-73.905471,Sandwich Place
5,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
6,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
7,Marble Hill,40.876551,-73.91066,El Malecon,40.879338,-73.904457,Caribbean Restaurant
8,Marble Hill,40.876551,-73.91066,The Bronx Public,40.878377,-73.903481,Pub
9,Marble Hill,40.876551,-73.91066,Estrellita Poblana V,40.879687,-73.906257,Mexican Restaurant


Let's check how many venues were returned for each neighborhood

In [188]:
manhattan_venues.groupby('Neighborhood')

In [189]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,100,100,100,100,100,100
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,100,100,100,100,100,100
East Village,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [190]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 308 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [191]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()


,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Szechuan Restaurant,TV Station,Taco Place,Taiwanese Restaurant,Tapas Restaurant,

And let's examine the new dataframe size.

In [192]:
manhattan_onehot.shape

(3979, 309)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [218]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Szechuan Restaurant,TV Station,Taco Place,Taiwanese Restaurant,Tapas Restaurant,

#### Let's confirm the new size

In [194]:
manhattan_grouped.shape

(40, 309)

#### Let's print each neighborhood along with the top 20 most common venues

In [195]:
num_top_venues = 20

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                    venue  freq
0             Coffee Shop  0.08
1                    Park  0.07
2               Wine Shop  0.04
3                   Hotel  0.03
4                   Plaza  0.03
5                     Gym  0.03
6               BBQ Joint  0.02
7            Burger Joint  0.02
8          Ice Cream Shop  0.02
9              Steakhouse  0.02
10              Bookstore  0.02
11   Gym / Fitness Center  0.02
12          Grocery Store  0.02
13  Performing Arts Venue  0.02
14          Shopping Mall  0.02
15             Kids Store  0.02
16                 Market  0.02
17         Scenic Lookout  0.02
18         Sandwich Place  0.02
19    American Restaurant  0.02


----Carnegie Hill----
                   venue  freq
0            Pizza Place  0.06
1            Coffee Shop  0.05
2            Yoga Studio  0.04
3                    Gym  0.04
4     Italian Restaurant  0.04
5             Art Museum  0.03
6                    Spa  0.03
7                 Bakery  0.03

                                       venue  freq
0                                    Theater  0.07
1                                      Hotel  0.05
2                               Dance Studio  0.05
3                       Gym / Fitness Center  0.04
4                         Italian Restaurant  0.04
5                                Music Venue  0.03
6                                Art Gallery  0.03
7                                Pizza Place  0.03
8                              Indie Theater  0.03
9                                        Gym  0.03
10                               Coffee Shop  0.03
11                       American Restaurant  0.03
12                                      Park  0.02
13                                 Wine Shop  0.02
14                                   Dog Run  0.02
15                                    Lounge  0.02
16                                 Hotel Bar  0.02
17                               Event Space  0.01
18                             

                   venue  freq
0                   Park  0.07
1       Sushi Restaurant  0.06
2            Coffee Shop  0.05
3            Pizza Place  0.04
4       Greek Restaurant  0.04
5          Deli / Bodega  0.03
6             Playground  0.02
7                    Bar  0.02
8           Cocktail Bar  0.02
9            Yoga Studio  0.02
10           Art Gallery  0.02
11          Tennis Court  0.02
12                 Hotel  0.02
13    Italian Restaurant  0.02
14    Mexican Restaurant  0.02
15                  Café  0.02
16  Caribbean Restaurant  0.01
17    Chinese Restaurant  0.01
18        History Museum  0.01
19            Club House  0.01


----Soho----
                       venue  freq
0             Clothing Store  0.05
1         Italian Restaurant  0.05
2                       Café  0.04
3                   Boutique  0.04
4          French Restaurant  0.04
5                Coffee Shop  0.04
6                Men's Store  0.03
7                 Shoe Store  0.03
8        American R

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [196]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 20 venues for each neighborhood.

In [316]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Wine Shop,Hotel,Gym,Plaza,Sandwich Place,Scenic Lookout,Mexican Restaurant,Shopping Mall,Kids Store,Food Truck,Bookstore,Market,Fountain,Burger Joint,BBQ Joint,Steakhouse,Performing Arts Venue,Grocery Store
1,Carnegie Hill,Pizza Place,Coffee Shop,Gym,Italian Restaurant,Yoga Studio,Bakery,Bookstore,Café,Cocktail Bar,Spa,Art Museum,Gym / Fitness Center,Wine Shop,Bar,Cosmetics Shop,French Restaurant,Gourmet Shop,Grocery Store,Hot Dog Joint,American Restaurant
2,Central Harlem,Southern / Soul Food Restaurant,Café,French Restaurant,African Restaurant,Seafood Restaurant,American Restaurant,Gym / Fitness Center,Sushi Restaurant,Theater,Chinese Restaurant,Coffee Shop,Pizza Place,Cosmetics Shop,Tapas Restaurant,Mexican Restaurant,Lounge,Yoga Studio,Jazz Club,Indian Restaurant,Wine Shop
3,Chelsea,Art Gallery,Coffee Shop,American Restaurant,Seafood Restaurant,Hotel,Nightclub,Italian Restaurant,Theater,Cupcake Shop,Bakery,Asian Restaurant,Salon / Barbershop,Bagel Shop,Tapas Restaurant,Ice Cream Shop,Wine Bar,Café,Cycle Studio,Donut Shop,Office
4,Chinatown,Chinese Restaurant,Cocktail Bar,Sandwich Place,Wine Bar,Ice Cream Shop,American Restaurant,Café,Speakeasy,Dumpling Restaurant,Optical Shop,Coffee Shop,Gourmet Shop,Mexican Restaurant,Thai Restaurant,Shoe Store,French Restaurant,History Museum,Pizza Place,Hotel,Asian Restaurant


<a id='item4'></a>

#### Creating one single column containing venues of all neighbourhoods for further Unique method application

In [314]:

df = pd.DataFrame(columns=['A'])
for k in range(20):
    if k == 0:
        string = str(k+1) + "st Most Common Venue"
        Venues = neighborhoods_venues_sorted[string]
    if k == 1:
        string = str(k+1) + "nd Most Common Venue"
        venues = neighborhoods_venues_sorted[string]
        Venues = Venues.append(venues)
    if k == 2:
        string = str(k+1) + "rd Most Common Venue"
        venues = neighborhoods_venues_sorted[string]
        Venues = Venues.append(venues)
    if k > 2:
        string = str(k+1) + "th Most Common Venue"
        venues = neighborhoods_venues_sorted[string]
        Venues = Venues.append(venues)
        
        




#### Finding unique venues among the dataset

In [313]:
unique = Venues.unique()
frame = pd.DataFrame({'Unique Venues' : [], 'Mark':[]})


for k in range(Venues.unique().shape[0]):
    frame.loc[k, 'Unique Venues'] = unique[k]
   
# Checking the shape of the new dataset
frame.shape



(148, 2)

## Getting the customers opinion on how important a specific venue is to him
#### More specifically, the customer is supposed to answer for each and every unique venue how important it is to him to have this venue in the 1km vicinity of its neighbourhood. This marking system will therefore calculate which neighbourhood fits best and how they are located on the map of Manhattan. 

#### For examiners who are not interested to check their optimum neighbourhood, they can skip the following coding cell and use the next coding cell to acquire random marks.

In [238]:
# Interested to see how this notebook really works? Give your opinion as described above for each of the following venues.
# If you run this cell, ignore the next cell. 
# If you decide to use random marks, run the next cell instead of this cell. 

for k in range(frame.shape[0]):    #

    answer = input("From 1 to 10, how important is it for you to have a ----" + str(frame.loc[k,"Unique Venues"]) + "---- nearby")
    frame.loc[k,"Mark"] = int(answer)
               
               

From 1 to 10, how important is it for you to have a ----Coffee Shop---- nearby9
From 1 to 10, how important is it for you to have a ----Pizza Place---- nearby7
From 1 to 10, how important is it for you to have a ----Southern / Soul Food Restaurant---- nearby1
From 1 to 10, how important is it for you to have a ----Art Gallery---- nearby2
From 1 to 10, how important is it for you to have a ----Chinese Restaurant---- nearby6
From 1 to 10, how important is it for you to have a ----French Restaurant---- nearby5
From 1 to 10, how important is it for you to have a ----Theater---- nearby2
From 1 to 10, how important is it for you to have a ----Bakery---- nearby9
From 1 to 10, how important is it for you to have a ----Cocktail Bar---- nearby3
From 1 to 10, how important is it for you to have a ----Gym---- nearby9
From 1 to 10, how important is it for you to have a ----American Restaurant---- nearby2
From 1 to 10, how important is it for you to have a ----Italian Restaurant---- nearby5
From 1 t

From 1 to 10, how important is it for you to have a ----Cheese Shop---- nearby7
From 1 to 10, how important is it for you to have a ----Playground---- nearby9
From 1 to 10, how important is it for you to have a ----Market---- nearby9
From 1 to 10, how important is it for you to have a ----Moroccan Restaurant---- nearby1
From 1 to 10, how important is it for you to have a ----Memorial Site---- nearby1
From 1 to 10, how important is it for you to have a ----Israeli Restaurant---- nearby10
From 1 to 10, how important is it for you to have a ----Pool---- nearby8
From 1 to 10, how important is it for you to have a ----Dog Run---- nearby9
From 1 to 10, how important is it for you to have a ----Farmers Market---- nearby8
From 1 to 10, how important is it for you to have a ----Toy / Game Store---- nearby7
From 1 to 10, how important is it for you to have a ----Garden---- nearby9
From 1 to 10, how important is it for you to have a ----Fountain---- nearby9
From 1 to 10, how important is it for y

Randomly creating marks 

In [202]:
Random = (np.random.rand(frame.shape[0],1)*10).astype(int)
frame['Mark'] = Random

Checking what the frame looks like

In [268]:
frame.head()

,Unique Venues,Mark
0,Coffee Shop,9
1,Pizza Place,7
2,Southern / Soul Food Restaurant,1
3,Art Gallery,2
4,Chinese Restaurant,6


## Creating a 40x20 matrix with the marks instead of the category names

Let us first check what the neighborhoods_venues_sorted dataframe looks like.

In [317]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Wine Shop,Hotel,Gym,Plaza,Sandwich Place,Scenic Lookout,Mexican Restaurant,Shopping Mall,Kids Store,Food Truck,Bookstore,Market,Fountain,Burger Joint,BBQ Joint,Steakhouse,Performing Arts Venue,Grocery Store
1,Carnegie Hill,Pizza Place,Coffee Shop,Gym,Italian Restaurant,Yoga Studio,Bakery,Bookstore,Café,Cocktail Bar,Spa,Art Museum,Gym / Fitness Center,Wine Shop,Bar,Cosmetics Shop,French Restaurant,Gourmet Shop,Grocery Store,Hot Dog Joint,American Restaurant
2,Central Harlem,Southern / Soul Food Restaurant,Café,French Restaurant,African Restaurant,Seafood Restaurant,American Restaurant,Gym / Fitness Center,Sushi Restaurant,Theater,Chinese Restaurant,Coffee Shop,Pizza Place,Cosmetics Shop,Tapas Restaurant,Mexican Restaurant,Lounge,Yoga Studio,Jazz Club,Indian Restaurant,Wine Shop
3,Chelsea,Art Gallery,Coffee Shop,American Restaurant,Seafood Restaurant,Hotel,Nightclub,Italian Restaurant,Theater,Cupcake Shop,Bakery,Asian Restaurant,Salon / Barbershop,Bagel Shop,Tapas Restaurant,Ice Cream Shop,Wine Bar,Café,Cycle Studio,Donut Shop,Office
4,Chinatown,Chinese Restaurant,Cocktail Bar,Sandwich Place,Wine Bar,Ice Cream Shop,American Restaurant,Café,Speakeasy,Dumpling Restaurant,Optical Shop,Coffee Shop,Gourmet Shop,Mexican Restaurant,Thai Restaurant,Shoe Store,French Restaurant,History Museum,Pizza Place,Hotel,Asian Restaurant


In [249]:
# Exchaning the names with the respective marks given by the user.
for k in range(frame.shape[0]):
    neighborhoods_venues_sorted.replace(frame.loc[k,"Unique Venues"], frame.loc[k,"Mark"], inplace=True)

In [250]:
# Evaluating the newly created dataset
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Battery Park City,8,10,6,1,9,8,8,3,5,8,7,2,9,8,8,2,3,3,1,10
1,Carnegie Hill,6,8,9,5,6,9,9,7,4,6,2,9,6,7,1,5,2,10,4,4
2,Central Harlem,2,7,5,2,1,4,9,7,3,6,8,6,1,8,5,9,6,7,2,6
3,Chelsea,2,8,4,1,1,1,5,3,6,9,7,3,7,8,7,6,7,4,6,7
4,Chinatown,6,4,8,6,7,4,7,2,2,1,8,2,5,3,6,5,1,6,1,7


### Calculating the weighted mark, meaning that the level of "Commonness" of the venue affects the mark. 

In [259]:
# First we'll create an array that will allow different importances for how common the venue is. 
Factor = np.linspace(1.0, 0.1, num=20)

# Setting the neighbourhood to index, so it will not interfere with subsequent multiplication
neighborhoods_venues_sorted_indexed = neighborhoods_venues_sorted.set_index('Neighborhood')

# Transforming all data into integers
Neighborhoods_venues_sorted_indexed = neighborhoods_venues_sorted_indexed.apply(pd.to_numeric, errors='ignore')

# Multiplying the mark matrix with the Factor
Neighbourhoods_weighed = Neighborhoods_venues_sorted_indexed.mul(Factor, axis=1)

8

#### Let us have a look at the newly created dataframe now. 

In [261]:
Neighbourhoods_weighed.head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
Neighborhood,,,,,,,,,,,,,,,,,,,,
Battery Park City,8.0,9.526316,5.431579,0.857895,7.294737,6.105263,5.726316,2.005263,3.105263,4.589474,3.684211,0.957895,3.884211,3.073684,2.694737,0.578947,0.726316,0.584211,0.147368,1.0
Carnegie Hill,6.0,7.621053,8.147368,4.289474,4.863158,6.868421,6.442105,4.678947,2.484211,3.442105,1.052632,4.310526,2.589474,2.689474,0.336842,1.447368,0.484211,1.947368,0.589474,0.4
Central Harlem,2.0,6.668421,4.526316,1.715789,0.810526,3.052632,6.442105,4.678947,1.863158,3.442105,4.210526,2.873684,0.431579,3.073684,1.684211,2.605263,1.452632,1.363158,0.294737,0.6
Chelsea,2.0,7.621053,3.621053,0.857895,0.810526,0.763158,3.578947,2.005263,3.726316,5.163158,3.684211,1.436842,3.021053,3.073684,2.357895,1.736842,1.694737,0.778947,0.884211,0.7
Chinatown,6.0,3.810526,7.242105,5.147368,5.673684,3.052632,5.010526,1.336842,1.242105,0.573684,4.210526,0.957895,2.157895,1.152632,2.021053,1.447368,0.242105,1.168421,0.147368,0.7


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [274]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = Neighbourhoods_weighed

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 2, 2, 4, 2, 1, 4, 0, 4], dtype=int32)

### Creating a new dataframe "final_df" which will contain the final information, including cluster information.

In [275]:
Data = {'Neighborhood': neighborhoods_venues_sorted.iloc[:,0], "Kmeans Cluster": kmeans.labels_}
final_df = pd.DataFrame(data=Data)

final_df.head()

,Neighborhood,Kmeans Cluster
0,Battery Park City,4
1,Carnegie Hill,4
2,Central Harlem,2
3,Chelsea,2
4,Chinatown,4


## Joining between this new dataframe and the dataframe that contains the latitude and longitude data.

In [276]:
final_df = final_df.join(manhattan_data.set_index('Neighborhood'), on="Neighborhood")


final_df.head() # check the last columns!

,Neighborhood,Kmeans Cluster,Borough,Latitude,Longitude
0,Battery Park City,4,Manhattan,40.711932,-74.016869
1,Carnegie Hill,4,Manhattan,40.782683,-73.953256
2,Central Harlem,2,Manhattan,40.815976,-73.943211
3,Chelsea,2,Manhattan,40.744035,-74.003116
4,Chinatown,4,Manhattan,40.715618,-73.994279


## Visualizing this in a Folium Map

In [277]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_df['Latitude'], final_df['Longitude'], final_df['Neighborhood'], final_df['Kmeans Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Calculating the average score of each neighbourhood and adding it to the final_df dataframe

In [311]:
temp = Neighbourhoods_weighed.mean(axis=1).reset_index()
final_df['Mean Score'] = temp.iloc[:,1]
final_df.head()

,Neighborhood,Kmeans Cluster,Borough,Latitude,Longitude,Mean Score
0,Battery Park City,4,Manhattan,40.711932,-74.016869,3.498684
1,Carnegie Hill,4,Manhattan,40.782683,-73.953256,3.534211
2,Central Harlem,2,Manhattan,40.815976,-73.943211,2.689474
3,Chelsea,2,Manhattan,40.744035,-74.003116,2.475789
4,Chinatown,4,Manhattan,40.715618,-73.994279,2.664737


#### Calculating the average score for each cluster, getting an idea which cluster fits best to the customers preferences. In this case, cluster 4, and then cluster 3, appears to be most fitting cluster for the customer. 

In [310]:
final_df.groupby('Kmeans Cluster').mean()

,Latitude,Longitude,Mean Score
Kmeans Cluster,,,
0,40.761897,-73.972212,3.005816
1,40.765382,-73.977923,3.059591
2,40.759597,-73.974971,2.865301
3,40.766536,-73.979986,3.138816
4,40.760680,-73.974500,3.290342


# Thank you for checking this notebook and I hope you enjoyed it!